In [1]:
from jupyter_dash import JupyterDash
import flask
import os
from random import randint
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import plotly.express as px
import dataextract
from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt
from datetime import time as tt
import zipfile

#server = flask.Flask(__name__)
#server.secret_key = os.environ.get('secret_key', str(randint(0, 1000000)))
app = JupyterDash(__name__)
app.title='Nashville Occupancy Dashboard'

In [2]:
mapbox_access_token = "pk.eyJ1Ijoidmlzb3ItdnUiLCJhIjoiY2tkdTZteWt4MHZ1cDJ4cXMwMnkzNjNwdSJ9.-O6AIHBGu4oLy3dQ3Tu2XA"
px.set_mapbox_access_token(mapbox_access_token)

In [3]:
df = dataextract.decompress_pickle('nashville_bus_occupancy_dashboard.pbz2')

In [4]:
routes=df.route_id.unique() 

In [5]:
def select_data(routes,timemin,timemax,startdate,enddate,months,weekdays):
    
    hourmax=int(timemax)
    if hourmax==24:
        endtime=tt(23,59,59)    
    else:
        minutesmax=int(60*(timemax-hourmax))
        endtime=tt(hourmax,minutesmax,0)
        
    hourmin=int(timemin)
    minutesmin=int(60*(timemin-hourmin))
    starttime=tt(hourmin,minutesmin,0)
    if months is None or len(months)==0:
        month_condition=True
    else:
        month_condition = ((df['month'].isin(months)))
    datacondtition=((df['datetime'] >= startdate) & (df['datetime'] <= enddate))    
    time_condition = ((df['timeofday'] >= starttime) & (df['timeofday'] <= endtime))
    if routes is None or len(routes)==0:
        route_condition=(True)
    else:
        route_condition  = ((df['route_id']).isin(routes))
    if weekdays is None or len(weekdays)==0:
        weekday_condition=(True)
    else:
        weekday_condition= ((df['dayofweek']).isin(weekdays))
    occupancy_condition=(df['occupancy'] >= 0)
    result = df.loc[occupancy_condition & month_condition & route_condition & time_condition  & weekday_condition]
    return result
    
    
    
    
    

In [6]:
def find_maximum_by_stop(dataframe):
    result=dataframe.groupby(['trip_start_time','route_id','direction_desc','year','month','day','stop_id','stop_sequence','stop_name','stop_lat','stop_lon'], as_index=False)['occupancy'].max()
    return result
    

In [7]:
def find_maximum_by_trip(dataframe):
    result=dataframe.groupby(['trip_start_time','route_id','direction_desc','year','month','day'], as_index=False)['occupancy'].max()
    return result

In [8]:
#result=find_maximum_by_stop(select_data(None,0,10.5,1,10,None))
#result=find_maximum_by_trip(select_data(None,0,10,1,10,1,30,None))

In [9]:
#result.head()

In [10]:

# Layout of Dash App
app.layout = html.Div(
    children=[
        html.Div(
            className="row",
            children=[
                # Column for user controls
                html.Div(
                    className="four columns div-user-controls",
                    children=[
                        dcc.Markdown('''# [SmartTransit.ai](https://smarttransit.ai) | Transit Occupancy'''),
                        html.Div(
                            className="div-for-dropdown",
                            children=[
                                #html.P("""Select Start Date """,style={'text-align': 'left' ,'font-weight':'bold'}),
                                dcc.Markdown('''## Select Start Date'''),
                                dcc.DatePickerSingle(
                                    id="date-picker",
                                    min_date_allowed=dt(2020, 1, 1),
                                    max_date_allowed=dt(2020, 6, 30),
                                    initial_visible_month=dt(2020, 1, 1),
                                    date=dt(2020, 1, 1).date(),
                                    display_format="MMMM D, YYYY",
                                    style={"border": "0px solid black"},
                                )
                            ],
                        ),
                        html.Div(
                            className="div-for-dropdown",
                            children=[                                
                                dcc.Markdown('''## Select End Date'''),
                                dcc.DatePickerSingle(
                                    id="date-picker-end",
                                    min_date_allowed=dt(2020, 1, 1),
                                    max_date_allowed=dt(2020, 6, 30),
                                    initial_visible_month=dt(2020, 6, 30),
                                    date=dt(2020, 6, 30).date(),
                                    display_format="MMMM D, YYYY",
                                    style={"border": "0px solid black"},
                                )
                            ],
                        ),
                        html.Div(
                            className="div-for-dropdown",
                            children=[                                
                                dcc.Markdown('''## Select Bus Routes'''),
                                dcc.Checklist(
                                        options=[
                                             {'label': '{}'.format(str(i)), 'value': '{}'.format(str(i))} for i in routes
                                        ],
                                        value=[''],
                                        id='bus-routes' ,
                                         labelStyle={'display': 'inline-block'} ,
                                    ),                                                         
                            ],
                        ),
                        # Change to side-by-side for mobile layout
                        html.Div(
                                    className="div-for-dropdown",
                                    children=[
                                        dcc.Markdown('''## Filter Data By Month'''),
                                        #html.P("""Select from the list to filter data by month.""",style={'text-align': 'left' ,'font-weight':'bold'}),
                                        # Dropdown to select times
                                        dcc.Checklist(
                                            id="bar-selector",
                                            options=[
                                                
                                                    {'label': 'Jan', 'value': '1'},
                                                    {'label': 'Feb', 'value': '2'},
                                                    {'label': 'Mar', 'value': '3'},
                                                    {'label': 'Apr', 'value': '4'},
                                                    {'label': 'May', 'value': '5'},
                                                    {'label': 'June', 'value': '6'},
                                                    {'label': 'July', 'value': '7'},
                                                    {'label': 'Aug', 'value': '8'},
                                                    {'label': 'Sep', 'value': '9'},
                                                    {'label': 'Oct', 'value': '10'},
                                                    {'label': 'Nov', 'value': '11'},
                                                    {'label': 'Dec', 'value': '12'},
                                                
                                                
                                            ],
                                            
                                            labelStyle={'display': 'inline-block'}                                         
                                        )
                                    ],
                                ),
                        html.Div(style={'display': 'none'},
                                    className="div-for-dropdown",                                    
                                    children=[
                                        dcc.Markdown('''## Filter by Pattern'''),
                                        dcc.Checklist( id='severity-basis',
                                                options=[                                             
                                                    {'label': 'Weekday', 'value': 'weekday'},
                                                    {'label': 'Weekend', 'value': 'weekend'},
                                                    ],
                                                value=['weekday','weekend'],
                                                labelStyle={'display': 'inline-block'} ,                                                     
                                            ),                                        
                                    ]
                                ),
                        html.Div(className="div-for-dropdown",
                                    children=[
                                    dcc.Markdown('''## Filter Data By Trip Start Time'''),
                                    #html.P("""Select Time Range""",style={'text-align': 'left' ,'font-weight':'bold'}),
                                    dcc.RangeSlider(
                                        id='time-slider',
                                        min=0,
                                        max=24,
                                        step=0.25,
                                        value=[0, 24],
                                        marks={i: '{}:00'.format(str(i).zfill(2)) for i in range(0, 25,4)},
                                    ),
                                    #dcc.Markdown('''##  Filter by Response Time (min).''') ,
                                    html.Div(style={'text-align': 'center','display':'none'},children=[dcc.Slider(
                                                                id='responsetime-value',
                                                                min=0,
                                                                max=70,
                                                                step=0.5,
                                                                marks={i: '{}'.format(i) for i in range(0, 70,10)},
                                                                value=0
                                                            ),],),    
                                                ]
                                    ),
                                    html.Div(style={'text-align': 'center','display':'none'}, children=[html.P('Incidents',id='incident-text',style={'text-align': 'left','font-weight': 'bold'}),
                                                       html.P('Months',id='month-text',style={'text-align': 'left','font-weight': 'bold'}),
                                                       html.P('Time',id='time-text',style={'text-align': 'left','font-weight': 'bold'}),
                                                       html.P('Response',id='response-text',style={'text-align': 'left','font-weight': 'bold'}),
                                                      ],
                                    )
                    ],
                ),
                # Column for app graphs and plots
                html.Div(
                    className="eight columns div-for-charts bg-grey",
                    children=[
                        
                        dcc.Markdown('''Adjust Slider below to configure the heatmap intensity.''') ,
                        html.Div(children=[dcc.Slider(
                                                    id='map-graph-radius',
                                                    min=1,
                                                    max=10,
                                                    step=0.05,
                                                    marks={i: '{}'.format(i) for i in range(1, 11)},
                                                    value=2
                                                ),],),    
                        dcc.Graph(id="map-graph"),                        
                        dcc.RadioItems( id='histogram-basis',
                                          options=[                                             
                                              {'label': 'Group By Month', 'value': 'month'},
                                              {'label': 'Group By Day of The Week', 'value': 'day'},
                                              {'label': 'Group By Hour of The Day', 'value': 'hour'},
                                              {'label': 'Show Response Time', 'value': 'response'}],
                                        labelStyle={'display': 'inline-block'} ,     
                                        value='month',style={'text-align': 'center'},
                                    ),
                        #html.Div(className="div-for-dropdown", children=[html.P(id='heatmap-text',style={'text-align': 'center'})]),                       
                        dcc.Graph(id="histogram"),
                    ],
                ),
            ],
        ),
        html.Div(
            className="row",
            children=[html.Div(className="div-for-dropdown", style={'text-align': 'center','display':'none'},
                                            children=[
                                                dcc.Markdown('Site designed by [ScopeLab from Vanderbilt University](http://scopelab.ai/) starting from [the Uber Ride Demo from Plotly](https://github.com/plotly/dash-sample-apps/tree/master/apps/dash-uber-rides-demo). Data source: Nashville Fire Department. Funding for this work has been provided by the National Science Foundation under awards [CNS-1640624](https://www.nsf.gov/awardsearch/showAward?AWD_ID=1640624) and  [IIS-1814958](https://www.nsf.gov/awardsearch/showAward?AWD_ID=1814958). Any opinions, findings, and conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the National Science Foundation.')]),                                   
            ]
        ),
    ]
)



In [11]:
# %%
@app.callback(
    Output('map-graph', 'figure'),
    [Input("date-picker", "date"),
    Input("date-picker-end", "date"),
    Input("map-graph-radius", "value"),
    Input('bus-routes', 'value'),
    Input("bar-selector", "value"),
    Input("time-slider", "value")]
)
def update_map_graph(start_date, end_date, radius, busroutes, datemonth, timerange):
    timemin,timemax=timerange
    #start_date,end_date
    #bus-routes
    latInitial=36.16228
    lonInitial=-86.774372
    result=select_data(busroutes,timemin,timemax,start_date,end_date,None,None)
    
    fig = go.Figure(go.Densitymapbox(lat=result['stop_lat'], lon=result['stop_lon'],z=result['occupancy'],
                radius=radius),layout=Layout(
                autosize=True,
                margin=go.layout.Margin(l=0, r=35, t=0, b=0),
                showlegend=False,
                mapbox=dict(
                    accesstoken=mapbox_access_token,
                    center=dict(lat=latInitial, lon=lonInitial),  # 40.7272  # -73.991251
                    style="light",
                    bearing=0,
                    zoom=10,
                ),
                updatemenus=[
                    dict(
                        buttons=(
                            [
                                dict(
                                    args=[
                                        {
                                            "mapbox.zoom": 10,
                                            "mapbox.center.lon": lonInitial,
                                            "mapbox.center.lat": latInitial,
                                            "mapbox.bearing": 0,
                                            "mapbox.style": "light",
                                        }
                                    ],
                                    label="Reset Zoom",
                                    method="relayout",
                                )
                            ]
                        ),
                        direction="left",
                        pad={"r": 0, "t": 0, "b": 0, "l": 0},
                        showactive=False,
                        type="buttons",
                        x=0.45,
                        y=0.02,
                        xanchor="left",
                        yanchor="bottom",
                        bgcolor="#1E1E1E",                    
                        borderwidth=1,
                        bordercolor="#6d6d6d",
                        font=dict(color="#FFFFFF"),
                    ),
                ],
            ),
            )
    return fig


In [12]:
#if __name__ == '__main__':
app.run_server(host='0.0.0.0', port=8080, debug=True, use_reloader=False)  
    #app.server.run(threaded=True)

Dash app running on http://0.0.0.0:8080/


/Users/abhishek/opt/anaconda3/envs/transit_dashboard/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
